In [1]:
from importlib import reload

import datasets
import numpy as np
import torch

import src.models.components.feature_extractor_dinov2
import src.models.components.sign_language_net
import src.models.components.spatiotemporal_encoder

In [2]:
rwth_phoenix = datasets.load_dataset('lukasbraach/rwth_phoenix_weather_2014', 'multisigner', streaming=True)
it = iter(rwth_phoenix['train'])
first = next(it)

first

{'tokens': ['__ON__',
  'LIEB',
  'ZUSCHAUER',
  'ABEND',
  'WINTER',
  'GESTERN',
  'loc-NORD',
  'SCHOTTLAND',
  'loc-REGION',
  'UEBERSCHWEMMUNG',
  'AMERIKA',
  'IX'],
 'frames': [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=210x260>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=210x260>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=210x260>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=210x260>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=210x260>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=210x260>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=210x260>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=210x260>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=210x260>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=210x260>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=210x260>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=210x260>,
  <PIL.PngImagePlugin.PngImageFile ima

In [3]:
reload(src.models.components.feature_extractor_dinov2)
from src.models.components.feature_extractor_dinov2 import SignLanguageFeatureExtractor

feature_extractor = SignLanguageFeatureExtractor()

src_tensor = [
    np.random.rand(3, 3, 224, 224),
    np.random.rand(1, 3, 224, 224),
    np.random.rand(6, 3, 224, 224),
]
batched_feature = feature_extractor(first['frames'], sampling_rate=25, pad_to_multiple_of=16, return_tensors="pt")

input_values = batched_feature.input_values
attention_mask = batched_feature.attention_mask

print("input_values:", input_values.shape)
print("attention_mask:", attention_mask)

input_values: torch.Size([1, 176, 768])
attention_mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]], dtype=torch.int32)


In [4]:
reload(src.models.components.spatiotemporal_encoder)
from src.models.components.spatiotemporal_encoder import SpatiotemporalEncoder
from src.models.components.spatiotemporal_encoder import SpatiotemporalEncoderConfig

config = SpatiotemporalEncoderConfig()

model = SpatiotemporalEncoder(config)
encoded = model(input_values, attention_mask)

encoded

Wav2Vec2BaseModelOutput(last_hidden_state=tensor([[[-1.6662, -1.1935, -1.2154,  ..., -0.1806, -0.1634, -0.5926],
         [-0.1440, -0.5459,  0.0645,  ..., -0.1463,  2.5808, -0.1304],
         [-1.1576, -0.5484, -1.0633,  ..., -0.5433,  0.3422, -0.0436],
         ...,
         [-1.1972, -0.0455, -0.9794,  ..., -0.9758, -0.3754,  0.4545],
         [-1.1595, -0.9980, -0.5680,  ...,  0.1559, -0.3439,  0.0621],
         [-1.9314, -1.1435, -1.3820,  ...,  0.6837, -0.5764,  1.7082]]],
       grad_fn=<NativeLayerNormBackward0>), extract_features=tensor([[[-0.4891, -0.3340, -1.2171,  ..., -0.1401, -1.6476, -0.0252],
         [ 0.9135,  0.3016,  0.4989,  ...,  0.8663,  0.9258,  0.4208],
         [ 0.6746, -1.0303,  0.1279,  ..., -0.8164, -1.4375, -0.2067],
         ...,
         [ 0.0723, -0.9306, -0.2125,  ..., -0.2989, -1.8556,  0.0058],
         [ 0.0261, -0.6040, -0.3050,  ...,  0.0639, -1.6045,  0.5867],
         [-0.2460, -0.3531, -0.7720,  ..., -0.1185, -1.2942,  0.2753]]],
       grad_f

In [29]:
from transformers import PreTrainedTokenizerFast

reload(src.models.components.sign_language_net)
from src.models.components.sign_language_net import SignLanguageNet

model = SignLanguageNet(tokenizer=PreTrainedTokenizerFast(
    pad_token="__PAD__",
    bos_token="__ON__",
    eos_token="__OFF__",
    unk_token="__UNK__",
    tokenizer_file="../src/etc/rwth_phoenix_tokenizer.json"
))

model

SignLanguageNet(
  (encoder): SpatiotemporalEncoder(
    (feature_extractor): SpatialFeatureEncoder()
    (feature_projection): SpatiotemporalFeatureProjection(
      (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Wav2Vec2Encoder(
      (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
        (conv): Conv1d(768, 768, kernel_size=(128,), stride=(1,), padding=(64,), groups=16)
        (padding): Wav2Vec2SamePadLayer()
        (activation): GELUActivation()
      )
      (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (layers): ModuleList(
        (0-5): 6 x Wav2Vec2EncoderLayer(
          (attention): Wav2Vec2Attention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=7

In [42]:
out = model.forward(input_values=input_values, attention_mask=attention_mask, labels=torch.LongTensor([[2, 2]]))

print(out.logits.shape)
pred = torch.argmax(out.logits, dim=2)

print(pred)

torch.Size([1, 2, 1935])
tensor([[3, 2]])
